In [26]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pickle
import time
import pandas as pd
import datetime

In [18]:
browser = webdriver.Chrome()
browser = webdriver.Chrome(executable_path=".\chromedriver.exe")
browser.get('https://www.hangseng.com/en-hk/home/')
chatbot_Button = browser.find_element('id',"floatingHaroBtn")
chatbot_Button.click()
print('Please click and process the newbie tutor mannually...\nAfter that Please call save_cookies() function...')

Please click and process the newbie tutor mannually...
After that Please call save_cookies() function...


In [14]:
'''
利用该方法去保存cookies 在pickle 文件
'''
cookies = browser.get_cookies()
print('cookies feteched...\n')
with open('cookies.pkl', 'wb') as f:
    pickle.dump(cookies, f)
print('cookies saving as ./cookies.pkl ... \nPlease go to check it out')
browser.quit()

cookies feteched...

cookies saving as ./cookies.pkl ... 
Please go to check it out


In [23]:
def start_off(csv_path):
    result=[]
    unfinished = []
    unfinished_error = []
    '''
    只提供csv的名字就好了，保存在同级目录
    '''
   
    browser = webdriver.Chrome(executable_path=".\chromedriver.exe")
    ###开网站
    browser.get('https://www.hangseng.com/en-hk/home/')
    ###加载cookies
    with open('./cookies.pkl', 'rb') as f:
        cookies = pickle.load(f)

    counter = 0
    for cookie in cookies:
        # print(cookie)
        browser.add_cookie(cookie)
        counter+=1 
        print(f'It has {counter} cookies were added',end='\r')

    browser.get('https://www.hangseng.com/en-hk/home/')

    print(f'It is {browser.title} now\nRedirecting...')

    ## 再次激活chatbot
    chatbot_Button = browser.find_element('id',"floatingHaroBtn")
    chatbot_Button.click()

    ###处理转跳页面
    handles = browser.window_handles
    browser.switch_to.window(handles[-1])

    print(f'It is {browser.title} now')
    ###准备问题数据集
    df = pd.read_csv(f'./{csv_path}.csv')

    ###因为懒，所以写了嵌套，将就着用
    def __cralwer_for_haro(task):

        ##等待textarea的元素load出来
        textarea = WebDriverWait(browser, 30).until(
            EC.visibility_of_element_located((By.ID, "userInput"))
        )
        ##输入问题
        textarea.send_keys(task)
        ##确认发送键是否存在
        send_button = WebDriverWait(browser, 30).until(
            EC.visibility_of_element_located((By.ID, "user-input-icon"))
        )
        ##发送问题
        send_button.click() 

        ##缓5秒等待它更新最新的回复
        time.sleep(5)

        answer = WebDriverWait(browser, 30).until(
            EC.visibility_of_all_elements_located((By.TAG_NAME, "p"))
        )
        
        answer_updated = answer[-1].get_attribute('innerText')
        
        result = {
            'question':task,
            'answer':answer_updated
        }

        return result 

    ###循环数据集输入
    counter =0
    for index, row in df.iterrows():
        que = row['question']
        try:
            res = __cralwer_for_haro(que)
            result.append(res)
            counter+=1
            print(counter,end='\r')
        except Exception as e:
            unfinished.append(que)
            unfinished_error.append(e)

    print('closing broswers...\n')
    browser.quit()

    print('finished all the tasks\nsaving to csv')
    que = []
    ans = []
    
    for i in result:
        que.append(i['question'])
        ans.append(i['answer'])

    Q_and_A = {
        'question':que,
        'answer':ans
    }
    data=pd.DataFrame(Q_and_A)
    file_name = f"{csv_path}_{datetime.date.today().strftime('%y%m%d')}"

    data.to_csv(f'./{file_name}.csv', index = False)

    unfinished_dict = {
        'question':unfinished,
        'error': unfinished_error
    }

    unfinished_data = pd.DataFrame(unfinished_dict)
    unfinished_data.to_csv(f'./{file_name}_unfinished',index = False)
    
    print(f'The result was been saved at ./{file_name}.csv')



In [18]:
#start_off('que')

C:\Users\lukemai\AppData\Local\Temp\ipykernel_25964\679682692.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=".\chromedriver.exe")


It is Hang Seng Bank Limited now
Redirecting...
It is  now
closing broswers...

finished all the tasks
saving to csv
The result was been saved at ./que+221207.csv


In [24]:
def split_tasks(csv_path, x):
    '''
     x 是指任务要分成多少份
    '''
    ##临时保存保存问题集，等待切分
    temp = []
    temp_file_name = []
    ##读任务
    df = pd.read_csv(f"{csv_path}.csv")

    for index,row in df.iterrows():
        temp.append(row['question'])

    ##懒人，嵌套函数
    def split(a, n):
        k, m = divmod(len(a), n)
        return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))
    
    tasks_x = list(split(temp,x))

    print('starting split tasks ...')
    counter = 0
    for i in tasks_x:
        d = {
            'question':i
        }
        data = pd.DataFrame(d)
        counter+=1
        file_name = f'{csv_path}_{counter}'
        data.to_csv(f'./{file_name}.csv',index=False)
        temp_file_name.append(file_name)
        print(f'{file_name} -- is ready!')
    
    print(temp_file_name)

    for i in temp_file_name:
        print(i)
        start_off(i)
    print('done! go to check your work!')

In [44]:
split_tasks('que', 10)

starting split tasks ...
que_1 -- is ready!
que_2 -- is ready!
que_3 -- is ready!
que_4 -- is ready!
que_5 -- is ready!
que_6 -- is ready!
que_7 -- is ready!
que_8 -- is ready!
que_9 -- is ready!
que_10 -- is ready!
['que_1', 'que_2', 'que_3', 'que_4', 'que_5', 'que_6', 'que_7', 'que_8', 'que_9', 'que_10']
que_1
It is Hang Seng Bank Limited now
Redirecting...
It is  now
closing broswers...

finished all the tasks
saving to csv
The result was been saved at ./que_1_221208.csv
que_2
It is Hang Seng Bank Limited now
Redirecting...
It is  now
closing broswers...

finished all the tasks
saving to csv
The result was been saved at ./que_2_221208.csv
que_3
It is Hang Seng Bank Limited now
Redirecting...
It is HARO now
closing broswers...

finished all the tasks
saving to csv
The result was been saved at ./que_3_221208.csv
que_4
It is Hang Seng Bank Limited now
Redirecting...
It is  now
closing broswers...

finished all the tasks
saving to csv
The result was been saved at ./que_4_221208.csv
que_